# Transfer Learning Demo

### This demo shows how to classifiy a dataset which contains real life images of the dogs and cats.

The demo shows step by step how to use top of the art pre-trained model (visual recognition modles) to classify binary dogs/cats domain with highest accuracy.
Firstly we show why using pre-trained model "as is" doesn't work well.
Secondly, we show how to retrain pre-trained model effectivly to reach the top of the art high score.   

### Step 0: Download datasets

In [ ]:
!pip install wget

In [ ]:
# top level directory
root_dir = "transfer_learning_demo"

!rm -rf $root_dir

# create directories
!mkdir $root_dir
!cd $root_dir; mkdir images

# download dataset
!cd $root_dir; wget https://raw.githubusercontent.com/Yura32000/practicals/master/data.zip -O data.zip 
!cd $root_dir; unzip -q data.zip  

# download transfer learning image
!cd $root_dir; wget https://raw.githubusercontent.com/Yura32000/practicals/master/transfer_learning.jpg
    
# inspect on disk
!pwd; ls -lah $root_dir; 

### Step 1: Import dependencies

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import wget

from keras.applications import ResNet50, imagenet_utils
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l2

### Step 2: Create Model

#### Create pre-trained model for classification task. Use weights learned using ImageNet dataset
See ResNet50 documentation at: https://keras.io/applications/#resnet50

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224,224,3))

#### Before training a model, you need to configure the learning process, which is done via the compile method.
See documentation at: https://keras.io/getting-started/sequential-model-guide/#compilation

### Step 3. Compile model

In [ ]:
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#resnet_model.summary()

### Step 4: Create image generators to feed trainig and test data from disk

#### Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely.
See documentation at: https://keras.io/preprocessing/image/#imagedatagenerator


In [ ]:
batch_size = 8

# each training image will be read from disk and transformed randomly according to generator policy. 
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# Takes the path to a directory, and generates batches of augmented/normalized data. Yields batches indefinitely, in an infinite loop.
train_generator = train_datagen.flow_from_directory(
        root_dir+'/data/train',
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
# each test image will be read from disk without any sort of transformation.
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        root_dir+'/data/validation',
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='binary')

### Step 5: Define auxilary function to investigate accuracy of a model

In [ ]:
def predict(model, url=None, image_path=None, name="", should_decode=True):
    # define categories names according to directories in which dataset is separated
    categories = sorted(os.listdir(root_dir+"/data/train"))
    print("categories:", categories)
    
    image_path = root_dir+"/images/"+name if not image_path else root_dir+"/data/"+image_path
    
    if not os.path.isfile(image_path):
        wget.download(url, out=image_path)
        print("downloaded:", url, "as", name)
    
    print("path:", image_path)
    
    # load image and scale it according to a model required input shape
    orig_image = load_img(image_path, target_size=(224, 224))
    plt.imshow(orig_image)
    
    image = img_to_array(orig_image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)
    
    # return categories with probabilities
    predictions = model.predict(image)
    #print(predictions)
    
    index = np.argmax(predictions)
    print("index:", index)
    
    decode_preds = imagenet_utils.decode_predictions(predictions) if should_decode else categories[index]
        
    print("decoded prediction:", decode_preds)

### Step 6: Investigate real accuracy of the model 

In [ ]:
# predict on the cat image from dataset
predict(model=resnet_model, image_path='train/cats/cat.207.jpg')

We can see that original ImageNet dataset contains a few cats categories rather then one category "cat".

In [ ]:
# predict on the dog image from dataset
predict(model=resnet_model, image_path='train/dogs/dog.613.jpg')

Same issue as above.

In [ ]:
# predict on the odg image from internet
predict(model=resnet_model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbQR4USIM1PLxSJfSXapS9WMaYMIlv1N-8fZYA90ADWGIhS6Oh', name='husky_dog.jpg')

Same issue as above.

In [ ]:
# predict on the cat image from internet
predict(model=resnet_model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQvZlZZ2sHrxhewRJHftyVN_J9m_Rx4_OiADBn3Hg6DCpqydayUcA', name='hairless_cat.jpg')

Same issue as above.

In [ ]:
# predict on the dog image from internet
predict(model=resnet_model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgG8-eb4guR9UsYNn6ANeuFh2AiDEBHffkjpZO-qmcTOK9FYPNEA', name='zebra_dog.jpg')

Same issue as above.

#### Summary: we can see that a model verclassifies to or needs. We need a way to narrow a classification only to 2 categories CATS and DOGS

### Step 7: Build a model which reuses resnet50 and its weights

#### The following figure displays visually what is transfer learning concept.

In [ ]:
from IPython.display import Image
Image(filename=root_dir+'/transfer_learning.jpg') 

In [ ]:
warnings.filterwarnings("ignore")

# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
output = base_model.output

output = Flatten()(output)
output = Dense(1024, activation='relu')(output)
output = Dropout(0.4)(output)
output = Dense(2, kernel_regularizer=l2(.0005), activation='softmax')(output)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=output)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False
  
# tells Keras to stop training when loss didn't improve for 3 epochs
earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# tells Keras to save the best model's weights
checkpointer = ModelCheckpoint(filepath=root_dir+'/resnet50_cats_dogs.h5', verbose=1, save_best_only=True)

# tells Keras to update learning rate during training 
def schedule(epoch):
    if epoch < 15:
        return .002
    elif epoch < 28:
        return .0004
    else:
        return .0001

learningRateScheduler = LearningRateScheduler(schedule)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'], callbacks=[earlyStopping, checkpointer, learningRateScheduler])

#model.summary()

### Step 8: Retrain new model while only last layers are trained 
We can achieve state of the art accuracy with only few epochs. Training this model with extra epochs will reach accuracy above 98% 

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch= 2000 // batch_size,
        epochs=2,
        validation_data=test_generator,
        validation_steps=800 // batch_size)

### Step 9: Estimate pretrianed model's accuracy and predict on cats dogs images

In [ ]:
scores = model.evaluate_generator(test_generator,steps=20)
print("Accuracy = ", scores[1])

This is a state of the art accuracy on less than a few minutes training

In [ ]:
# predict on the cat image from dataset
predict(model=model, image_path='train/cats/cat.207.jpg', should_decode=False)

In [ ]:
# predict on the dog image from dataset
predict(model=model, image_path='train/dogs/dog.613.jpg', should_decode=False)

In [ ]:
# predict on the cat image from internet
predict(model=model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbQR4USIM1PLxSJfSXapS9WMaYMIlv1N-8fZYA90ADWGIhS6Oh', name='husky_dog.jpg', should_decode=False)

In [ ]:
# predict on the cat image from internet
predict(model=model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQvZlZZ2sHrxhewRJHftyVN_J9m_Rx4_OiADBn3Hg6DCpqydayUcA', name='hairless_cat.jpg', should_decode=False)

In [ ]:
# predict on the cat image from internet
predict(model=model, url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTgG8-eb4guR9UsYNn6ANeuFh2AiDEBHffkjpZO-qmcTOK9FYPNEA', name='zebra_dog.jpg', should_decode=False)

Oops: Misclassification. However we can see that realizing this image is very hard to classify

### Step 10: Save model in order to share it beween platforms or allow online training 

In [ ]:
from keras.models import load_model

path = root_dir + '/my_model.h5'

model.save(path)  # creates a HDF5 file 'my_model.h5'

# returns a compiled model
# identical to the previous one
new_model = load_model(path)

In [ ]:
scores = new_model.evaluate_generator(test_generator,steps=20)
print("Accuracy = ", scores[1])

In [ ]:
# show files on disk
!ls -lah $root_dir